In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from time import time
import re
import pickle

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
df = pd.read_csv('train.csv')
poi_street_df = df["POI/street"].str.split("/", n = 1, expand = True) 
df["POI"]= poi_street_df[0] 
df["street"]= poi_street_df[1] 
df1 = df.drop(["POI/street"], axis=1) 
df1

,id,raw_address,POI,street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,,jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",,
2,2,setu siung 119 rt 5 1 13880 cipayung,,siung
3,3,"toko dita, kertosono",toko dita,
4,4,jl. orde baru,,jl. orde baru
...,...,...,...,...
299995,299995,jend ahmad yani 331 kertasari ciamis,,jend ahmad yani
299996,299996,"raya cila kko, cilandak timur kel.",,raya cila kko
299997,299997,tanjung gusta jl. yaya 2 no 17,,
299998,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri,


In [4]:
def find_first_index(ra_split, st_split):
    
    if len(st_split) <= len(ra_split): # new
    
        num_iter = len(ra_split) - len(st_split) + 1
        overlap_list = []

        for i in range(num_iter):
            window = ra_split[i: i+len(st_split)]
            overlap = list(set(window) & set(st_split))
            overlap_list.append(len(overlap))

        max_overlap = [e for e in range(len(overlap_list)) if overlap_list[e] == max(overlap_list)]
        if len(max_overlap) == 1:
            return max_overlap[0]

        else:
            count_list = []
            for idx in max_overlap:
                subset_ra = ra_split[idx: idx+len(st_split)]
                count = 0
                for e in range(len(subset_ra)):
                    if subset_ra[e] not in st_split[e]:
                        count += 0
                    else:
                        count += 1
                count_list.append(count)
            index = count_list.index(max(count_list))
            return max_overlap[index]
    else:
        return 0

In [5]:
def fix_street_errors(row):
    
    raw_add = row['raw_address']

    # If a street name is extracted...
    if row['street'] != "":

        raw_add_split = word_tokenize(raw_add)
        
        extr_street = row['street']
        extr_street_split = word_tokenize(extr_street)
        
        # If the extracted street is in the raw address as an entire string, good!
        if extr_street in raw_add:
            return raw_add
        
        # This is where there are discrepancies!
        else:
            index_in_ra = find_first_index(raw_add_split, extr_street_split)
            raw_add_split[index_in_ra: index_in_ra+len(extr_street_split)] = extr_street_split
            updated_raw_add = ' '.join(raw_add_split).replace(' ,', ',').replace(' .', '.').replace(' )', ')').replace(' (', '(').replace(' ?', '?')          
            return updated_raw_add
      
    # If a street name is originally an empty string, we just assume there's no error. 
    else:
        return raw_add

In [6]:
start = time()

df1['cleaned_raw_add'] = df1.apply(fix_street_errors, axis=1)

print("Executed in {} minutes.".format(round((time() - start)/60, 3)))

# Sanity checks
df1.loc[[69, 86, 117, 130, 135, 169], :]

Executed in 0.963 minutes.


,id,raw_address,POI,street,cleaned_raw_add
69,69,cak 11 nagasari karawang barat,,cakrad,cakrad 11 nagasari karawang barat
86,86,simpang tiga kah nasu no 112 28284 bukit raya,,kaharu nasu,simpang tiga kaharu nasu no 112 28284 bukit raya
117,117,ahmad dah iv kukusan beji,,ahmad dahlan iv,ahmad dahlan iv kukusan beji
130,130,mangla rege no h1 3 cimekar kel. cileunyi,,manglayang rege,manglayang rege no h1 3 cimekar kel. cileunyi
135,135,"cipinang besar selatan lintas ibadah, cipi jay...",lintas ibadah,cipinang jaya 1a,"cipinang besar selatan lintas ibadah, cipinang..."
169,169,"taman mer, 13 electr laun system, 2 meruya utara",electrolux laundry system,taman meruya,"taman meruya, 13 electr laun system, 2 meruya ..."


In [7]:
def get_street_mapping_dict(row):
    
    raw_add = row['raw_address']

    # If a street name is extracted...
    if row['street'] != "":

        raw_add_split = word_tokenize(raw_add)
        
        extr_street = row['street']
        extr_street_split = word_tokenize(extr_street)
        
        # If the extracted street is in the raw address as an entire string, good!
        if extr_street in raw_add:
            return None
        
        # This is where there are discrepancies!
        else:
            index_in_ra = find_first_index(raw_add_split, extr_street_split)
            before = raw_add_split[index_in_ra: index_in_ra+len(extr_street_split)] 
            before = ' '.join(before)
            return before, extr_street
      
    # If a street name is originally an empty string, we just assume there's no error. 
    else:
        return None

In [8]:
start = time()

df1['street_mapping'] = df1.apply(get_street_mapping_dict, axis=1)

print("Executed in {} minutes.".format(round((time() - start)/60, 3)))

# Sanity checks
df1.loc[[69, 86, 117, 130, 135, 169], :]

Executed in 0.906 minutes.


,id,raw_address,POI,street,cleaned_raw_add,street_mapping
69,69,cak 11 nagasari karawang barat,,cakrad,cakrad 11 nagasari karawang barat,"(cak, cakrad)"
86,86,simpang tiga kah nasu no 112 28284 bukit raya,,kaharu nasu,simpang tiga kaharu nasu no 112 28284 bukit raya,"(kah nasu, kaharu nasu)"
117,117,ahmad dah iv kukusan beji,,ahmad dahlan iv,ahmad dahlan iv kukusan beji,"(ahmad dah iv, ahmad dahlan iv)"
130,130,mangla rege no h1 3 cimekar kel. cileunyi,,manglayang rege,manglayang rege no h1 3 cimekar kel. cileunyi,"(mangla rege, manglayang rege)"
135,135,"cipinang besar selatan lintas ibadah, cipi jay...",lintas ibadah,cipinang jaya 1a,"cipinang besar selatan lintas ibadah, cipinang...","(cipi jaya 1a, cipinang jaya 1a)"
169,169,"taman mer, 13 electr laun system, 2 meruya utara",electrolux laundry system,taman meruya,"taman meruya, 13 electr laun system, 2 meruya ...","(taman mer, taman meruya)"


In [9]:
# Create a separate dataframe containing only values in `street_mapping` columns, i.e., rows where changes occurred
df_with_street_mappings = df1[df1['street_mapping'].notnull()]

street_mapping_dict = dict()

# Create a mapping dictionary, where key is the truncated word/words and the value is the corresponding correct word/words
for row, col in df_with_street_mappings.iterrows():
    street_mapping_dict[col['street_mapping'][0]] = col['street_mapping'][1]
    
# How many street errors are there altogether?
print("Length of street mapping dictionary:", len(street_mapping_dict))

Length of street mapping dictionary: 10591


In [10]:
def fix_poi_errors(row):
    
    raw_add = row['cleaned_raw_add']

    # If a POI name is extracted...
    if row['POI'] != "":

        raw_add_split = word_tokenize(raw_add)
        
        extr_poi = row['POI']
        extr_poi_split = word_tokenize(extr_poi)
        
        # If the extracted POI is in the raw address as an entire string, good!
        if extr_poi in raw_add:
            return raw_add
        
        # This is where there are discrepancies!
        else:
            index_in_ra = find_first_index(raw_add_split, extr_poi_split)
            raw_add_split[index_in_ra: index_in_ra+len(extr_poi_split)] = extr_poi_split
            updated_raw_add = ' '.join(raw_add_split).replace(' ,', ',').replace(' .', '.').replace(' )', ')').replace(' (', '(').replace(' ?', '?')          
            return updated_raw_add
      
    # If a POI name is originally an empty string, we just assume there's no error. 
    else:
        return raw_add

In [11]:
start = time()

df1['cleaned_raw_add_1'] = df1.apply(fix_poi_errors, axis=1)

print("Executed in {} minutes.".format(round((time() - start)/60, 3)))

# Sanity checks
df1.loc[[10, 11, 40, 110, 152, 157, 169], :]

Executed in 0.556 minutes.


,id,raw_address,POI,street,cleaned_raw_add,street_mapping,cleaned_raw_add_1
10,10,"cikahuripan sd neg boj 02 klap boj, no 5 16877",sd negeri bojong 02,klap boj,"cikahuripan sd neg boj 02 klap boj, no 5 16877",None,"cikahuripan sd negeri bojong 02 klap boj, no 5..."
11,11,"yaya atohar,",yayasan atohariyah,,"yaya atohar,",None,"yayasan atohariyah,"
40,40,mar tabl metro iringmulyo metro timur,markaz tabligh metro,,mar tabl metro iringmulyo metro timur,None,markaz tabligh metro iringmulyo metro timur
110,110,"cv. hin oto kenc, sido ii, q 29",cv. hingdi oto kencana,sido ii,"cv. hin oto kenc, sido ii, q 29",None,"cv. hingdi oto kencana, sido ii, q 29"
152,152,"ujung harapan, no 48a pd. karya warga mand cim...",pd. karya warga mandiri cimb,ujung harapan,"ujung harapan, no 48a pd. karya warga mand cim...",None,"ujung harapan, no 48a pd. karya warga mandiri ..."
157,157,"alun - alun kuli nusan, kar",alun - alun kuliner nusantara,kar,"alun - alun kuli nusan, kar",None,"alun - alun kuliner nusantara, kar"
169,169,"taman mer, 13 electr laun system, 2 meruya utara",electrolux laundry system,taman meruya,"taman meruya, 13 electr laun system, 2 meruya ...","(taman mer, taman meruya)","taman meruya, 13 electrolux laundry system, 2 ..."


In [12]:
def get_poi_mapping_dict(row):
    
    raw_add = row['cleaned_raw_add']

    # If a POI name is extracted...
    if row['POI'] != "":

        raw_add_split = word_tokenize(raw_add)
        
        extr_poi = row['POI']
        extr_poi_split = word_tokenize(extr_poi)
        
        # If the extracted POI is in the raw address as an entire string, good!
        if extr_poi in raw_add:
            return None
        
        # This is where there are discrepancies!
        else:
            index_in_ra = find_first_index(raw_add_split, extr_poi_split)
            before = raw_add_split[index_in_ra: index_in_ra+len(extr_poi_split)] 
            before = ' '.join(before)
            return before, extr_poi
      
    # If a POI name is originally an empty string, we just assume there's no error. 
    else:
        return None

In [13]:
start = time()

df1['poi_mapping'] = df1.apply(get_poi_mapping_dict, axis=1)

print("Executed in {} minutes.".format(round((time() - start)/60, 3)))

# Sanity checks
df1.loc[[10, 11, 40, 110, 152, 157, 169], :]

Executed in 0.541 minutes.


,id,raw_address,POI,street,cleaned_raw_add,street_mapping,cleaned_raw_add_1,poi_mapping
10,10,"cikahuripan sd neg boj 02 klap boj, no 5 16877",sd negeri bojong 02,klap boj,"cikahuripan sd neg boj 02 klap boj, no 5 16877",None,"cikahuripan sd negeri bojong 02 klap boj, no 5...","(sd neg boj 02, sd negeri bojong 02)"
11,11,"yaya atohar,",yayasan atohariyah,,"yaya atohar,",None,"yayasan atohariyah,","(yaya atohar, yayasan atohariyah)"
40,40,mar tabl metro iringmulyo metro timur,markaz tabligh metro,,mar tabl metro iringmulyo metro timur,None,markaz tabligh metro iringmulyo metro timur,"(mar tabl metro, markaz tabligh metro)"
110,110,"cv. hin oto kenc, sido ii, q 29",cv. hingdi oto kencana,sido ii,"cv. hin oto kenc, sido ii, q 29",None,"cv. hingdi oto kencana, sido ii, q 29","(cv . hin oto kenc, cv. hingdi oto kencana)"
152,152,"ujung harapan, no 48a pd. karya warga mand cim...",pd. karya warga mandiri cimb,ujung harapan,"ujung harapan, no 48a pd. karya warga mand cim...",None,"ujung harapan, no 48a pd. karya warga mandiri ...","(pd . karya warga mand cimb, pd. karya warga m..."
157,157,"alun - alun kuli nusan, kar",alun - alun kuliner nusantara,kar,"alun - alun kuli nusan, kar",None,"alun - alun kuliner nusantara, kar","(alun - alun kuli nusan, alun - alun kuliner n..."
169,169,"taman mer, 13 electr laun system, 2 meruya utara",electrolux laundry system,taman meruya,"taman meruya, 13 electr laun system, 2 meruya ...","(taman mer, taman meruya)","taman meruya, 13 electrolux laundry system, 2 ...","(electr laun system, electrolux laundry system)"


In [14]:
# Create a separate dataframe containing only values in `poi_mapping` columns, i.e., rows where changes occurred
df_with_poi_mappings = df1[df1['poi_mapping'].notnull()]

poi_mapping_dict = dict()

# Create a mapping dictionary, where key is the truncated word/words and the value is the corresponding correct word/words
for row, col in df_with_poi_mappings.iterrows():
    poi_mapping_dict[col['poi_mapping'][0]] = col['poi_mapping'][1]
    
# How many POI errors are there altogether?
print("Length of POI mapping dictionary:", len(poi_mapping_dict))

Length of POI mapping dictionary: 43382


In [15]:
# Select required columns
cleaned_df = df1[['id','cleaned_raw_add_1', 'POI', 'street']]

# Rename columns
cleaned_df.columns = ['id', 'raw_address', 'POI', 'street']

# Sanity checks
cleaned_df.loc[[10, 11, 40, 69, 86, 110, 117, 130, 135, 152, 157, 169], :]

,id,raw_address,POI,street
10,10,"cikahuripan sd negeri bojong 02 klap boj, no 5...",sd negeri bojong 02,klap boj
11,11,"yayasan atohariyah,",yayasan atohariyah,
40,40,markaz tabligh metro iringmulyo metro timur,markaz tabligh metro,
69,69,cakrad 11 nagasari karawang barat,,cakrad
86,86,simpang tiga kaharu nasu no 112 28284 bukit raya,,kaharu nasu
110,110,"cv. hingdi oto kencana, sido ii, q 29",cv. hingdi oto kencana,sido ii
117,117,ahmad dahlan iv kukusan beji,,ahmad dahlan iv
130,130,manglayang rege no h1 3 cimekar kel. cileunyi,,manglayang rege
135,135,"cipinang besar selatan lintas ibadah, cipinang...",lintas ibadah,cipinang jaya 1a
152,152,"ujung harapan, no 48a pd. karya warga mandiri ...",pd. karya warga mandiri cimb,ujung harapan


In [16]:
# Load test dataset
test_df = pd.read_csv('test.csv')

# Preview
test_df.head()

,id,raw_address
0,0,s. par 53 sidanegara 4 cilacap tengah
1,1,"angg per, baloi indah kel. lubuk baja"
2,2,"asma laun, mand imog,"
3,3,"ud agung rej, raya nga sri wedari karanganyar"
4,4,"cut mutia, 35 baiturrahman"


In [17]:
new_street_mapping_dict = {k: v for k, v in street_mapping_dict.items() if len(k.split()) > 1}
new_poi_mapping_dict = {k: v for k, v in poi_mapping_dict.items() if len(k.split()) > 1}

print("Length of street mapping dictionary after removing single words:", len(new_street_mapping_dict))
print("Length of POI mapping dictionary after removing single words:", len(new_poi_mapping_dict))

Length of street mapping dictionary after removing single words: 9111
Length of POI mapping dictionary after removing single words: 41735


In [18]:
# Save the cleaned train dataset
cleaned_df.to_csv('cleaned_train.csv', index=False)

In [ ]:
start = time()

# Replace truncated words in raw_address of test set with correct street labels
for row, col in test_df.iterrows():
    for k, v in new_street_mapping_dict.items():
        if k in col['raw_address']:
            test_df.loc[row, 'raw_address'] = test_df.loc[row, 'raw_address'].replace(k, v)

# Replace truncated words in raw_address of test set with correct POI labels
for row, col in test_df.iterrows():
    for k, v in new_poi_mapping_dict.items():
        if k in col['raw_address']:
            test_df.loc[row, 'raw_address'] = test_df.loc[row, 'raw_address'].replace(k, v)

print("Executed in {} minutes.".format(round((time() - start)/60, 3)))

In [ ]:
# Save the cleaned test dataset
test_df.to_csv('cleaned_test.csv', index=False)

In [ ]:
# Save the dictionaries
s_file = open("street_mapping_dict.pkl", "wb")
pickle.dump(street_mapping_dict, s_file)
s_file.close()

p_file = open("poi_mapping_dict.pkl", "wb")
pickle.dump(street_mapping_dict, p_file)
p_file.close()

In [ ]:
s_file = open("street_mapping_dict.pkl", "rb")
street_mapping_dict = pickle.load(s_file)
print(street_mapping_dict)

p_file = open("poi_mapping_dict.pkl", "rb")
poi_mapping_dict = pickle.load(p_file)
print(poi_mapping_dict)